In [1]:
# Add Lincoln to system path
import sys
sys.path.append("/Users/seth/development/lincoln/")

In [2]:
from torch import Tensor
import torch

import typing
from typing import List, Tuple

In [3]:
import lincoln as lnc
from lincoln.layers import Dense
from lincoln.losses import LogSigmoidLoss, MeanSquaredError
from lincoln.optimizers import SGD
from lincoln.activations import Sigmoid, LogSigmoid
from lincoln.network import NeuralNetwork

## Regression network

In [4]:
from sklearn.datasets import load_boston

boston = load_boston()

data = boston.data
target = boston.target
features = boston.feature_names

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)

/Users/seth/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/seth/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Regression models

In [5]:
from lincoln.losses import MeanSquaredError
from lincoln.optimizers import SGD

from lincoln.activations import LinearAct, Sigmoid

In [6]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError(),
            optimizer = SGD())

lr.fit(X_train, y_train, X_test, y_test,  
       epochs = 10,
       eval_every = 1,
       seed=82618);

Validation loss after 1 epochs is 4404.868
Validation loss after 2 epochs is 3859.566
Validation loss after 3 epochs is 3753.522
Validation loss after 4 epochs is 3802.207
Validation loss after 5 epochs is 3977.285
Validation loss after 6 epochs is 3975.196
Validation loss after 7 epochs is 4758.660
Validation loss after 8 epochs is 4110.424
Validation loss after 9 epochs is 3970.611
Validation loss after 10 epochs is 4082.891


In [7]:
def mae(y_true, y_pred):
    return round(torch.mean(torch.abs(y_true - y_pred)).item(), 4)

def mse(y_true, y_pred):
    return round(torch.mean(torch.pow(y_true - y_pred, 2)).item(), 4)

In [8]:
def eval_regression_model(lr: NeuralNetwork, 
                          X_test: Tensor, 
                          y_test: Tensor):
    preds = lr.forward(X_test)
    preds = preds.reshape(preds.shape[0])
    return mae(preds, y_test), mse(preds, y_test)

In [9]:
eval_regression_model(lr, X_test, y_test)

(3.6518, 26.8611)

In [10]:
nn = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError(),
            optimizer = SGD())

nn.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618);

Validation loss after 10 epochs is 2421.626
Validation loss after 20 epochs is 1583.616
Validation loss after 30 epochs is 1485.388
Validation loss after 40 epochs is 1567.009
Validation loss after 50 epochs is 1532.060
Validation loss after 60 epochs is 1729.032
Validation loss after 70 epochs is 1768.350
Validation loss after 80 epochs is 1659.488
Validation loss after 90 epochs is 1612.138
Validation loss after 100 epochs is 1579.115


In [11]:
eval_regression_model(nn, X_test, y_test)

(2.2825, 10.3889)

In [12]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError(),
            optimizer = SGD())

dl.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618);

Validation loss after 10 epochs is 2225.604
Validation loss after 20 epochs is 2655.737
Validation loss after 30 epochs is 1803.951
Validation loss after 40 epochs is 1675.552
Validation loss after 50 epochs is 1384.854
Validation loss after 60 epochs is 1790.974
Validation loss after 70 epochs is 1282.833
Validation loss after 80 epochs is 1245.037
Validation loss after 90 epochs is 1302.036
Validation loss after 100 epochs is 1333.448


In [13]:
eval_regression_model(dl, X_test, y_test)

(2.0576, 8.7727)

## Classification network

In [14]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [15]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=82618)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)

In [16]:
logr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LogSigmoid())],
            loss = LogSigmoidLoss(),
            optimizer = SGD())

logr.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618);

NameError: name 'prediction' is not defined

Something off with trying to predict log probabilities here.

In [ ]:
logr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
            optimizer = SGD())

logr.fit(X_train, y_train, X_test, y_test,  
       epochs = 500,
       eval_every = 50,
       seed=82618);

In [ ]:
def accuracy(y_true, y_pred):
    return round(torch.sum(torch.eq(y_true, y_pred)).item() / y_true.size()[0], 4)

In [ ]:
def eval_classification_model(model: NeuralNetwork, 
                              X_test: Tensor, 
                              y_test: Tensor):
    preds = model.forward(X_test)
    preds = preds > 0.5
    preds = preds.reshape(preds.shape[0]).type(torch.FloatTensor)  
    return accuracy(preds, y_test)

In [ ]:
eval_classification_model(logr, X_test, y_test)

In [ ]:
logr_nn = NeuralNetwork(
    layers=[Dense(neurons=30, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
            optimizer = SGD())

logr_nn.fit(X_train, y_train, X_test, y_test,  
       epochs = 500,
       eval_every = 50,
       seed=82618);

In [ ]:
eval_classification_model(logr_nn, X_test, y_test)